In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import scipy
import seaborn as sns

In [2]:
#PROBLEM INSTANCE
instance = np.array([
    [0.9, 0.1, 0.1],
    [0.1, 0.9, 0.1],
    [0.1, 0.1, 0.9],
    [0.5, 0.5, 0.5]
])
delt=0.1
C=10

In [3]:
instance.shape

(4, 3)

In [4]:
def getreward(instance, arm, client):
    mean=instance[arm][client]
    reward=np.random.normal(loc=mean, scale=1, size=1)[0]
    return reward


def all_sets_empty(set_list):
    return all(not s for s in set_list)

In [90]:
def FEDELIM(instance,delt,C):
    K=instance.shape[0]
    M=instance.shape[1]
    answer=np.zeros((M+1))
    time=0
    emp_means_local=np.zeros((K,M))
    emp_means_global=np.zeros(K)
    local_contention=[]

    for i in range(M):
        local_contention.append(set(range(K)))
    global_contention=set(range(K))

    pulls=local_contention
    cost=0

    while len(global_contention) > 0 or any(len(s) > 0 for s in local_contention):
        time+=1
        conf_local=np.sqrt((2*np.log((8*K*M*time*time)/delt))/time)
        conf_global=np.sqrt((2*np.log((8*K*time*time)/delt))/(time*M))

        for m in range(M):
            pulls[m]=global_contention | local_contention[m]

            if len(pulls[m])>1:
                cost+=len(pulls[m])

                for k in pulls[m]:
                    rew=getreward(instance,k,m)
                    emp_means_local[k][m]=(emp_means_local[k][m]*(time-1)+rew)/time

            if len(local_contention[m])>1:
                local_best=max(emp_means_local[arm][m] for arm in local_contention[m])

                for k in set(local_contention[m]):

                    if local_best-emp_means_local[k][m]>=2*conf_local:
                        local_contention[m].remove(k)

            if len(local_contention[m])==1:
                answer[m]=local_contention[m].pop()
                
        if len(global_contention)>1 and (time & (time -1)) == 0:
            cost+=len(global_contention)*M*C

            for k in set(global_contention):
                emp_means_global[k]=np.mean(emp_means_local[k][:M])
            global_best=max(emp_means_global[arm] for arm in global_contention)

            for k in set(global_contention):

                if global_best-emp_means_global[k]>=2*conf_global:
                    global_contention.remove(k)

        if len(global_contention)==1:
            answer[M]=global_contention.pop()
        
    return (cost,answer,time)

In [92]:
np.random.seed(None)
cost,answer,time=FEDELIM(instance,delt,C)
print("total cost:", cost)
print(answer)

total cost: 99984
[0. 1. 2. 3.]


In [93]:
def FEDELIM0(instance,delt,C):
    K=instance.shape[0]
    M=instance.shape[1]
    answer=np.zeros((M+1))
    time=0
    emp_means_local=np.zeros((K,M))
    emp_means_global=np.zeros(K)
    local_contention=[]

    for i in range(M):
        local_contention.append(set(range(K)))
    global_contention=set(range(K))

    pulls=local_contention
    cost=0

    while len(global_contention) > 0 or any(len(s) > 0 for s in local_contention):
        time+=1
        conf_local=np.sqrt((2*np.log((8*K*M*time*time)/delt))/time)
        conf_global=np.sqrt((2*np.log((8*K*time*time)/delt))/(time*M))

        for m in range(M):
            pulls[m]=global_contention | local_contention[m]

            if len(pulls[m])>1:
                cost+=len(pulls[m])

                for k in pulls[m]:
                    rew=getreward(instance,k,m)
                    emp_means_local[k][m]=(emp_means_local[k][m]*(time-1)+rew)/time

            if len(local_contention[m])>1:
                local_best=max(emp_means_local[arm][m] for arm in local_contention[m])

                for k in set(local_contention[m]):

                    if local_best-emp_means_local[k][m]>=2*conf_local:
                        local_contention[m].remove(k)

            if len(local_contention[m])==1:
                answer[m]=local_contention[m].pop()
                
        if len(global_contention)>1:
            cost+=len(global_contention)*M*C

            for k in set(global_contention):
                emp_means_global[k]=np.mean(emp_means_local[k][:M])
            global_best=max(emp_means_global[arm] for arm in global_contention)

            for k in set(global_contention):

                if global_best-emp_means_global[k]>=2*conf_global:
                    global_contention.remove(k)

        if len(global_contention)==1:
            answer[M]=global_contention.pop()
        
    return (cost,answer,time)

In [120]:
np.random.seed(None)
cost,answer,time=FEDELIM0(instance,delt,C)
print("total cost:", cost)
print(answer)

total cost: 492756
[0. 3. 3. 3.]
